In [20]:
import gym
import numpy as np
import gym_donkeycar
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [21]:
import warnings
warnings.filterwarnings("ignore")

In [22]:
def flatten_image(img):
    return img.flatten()

def preprocessing(img,condition = True):
    if condition :
        img = img[120:,:,:]
        img = cv2.cvtColor(img,cv2.COLOR_RGB2YUV)
        img = cv2.GaussianBlur(img,(3,3),0)
        img = cv2.resize(img,(200,66))
        img = img/255
        return img
    else :
        return img


In [23]:
def draw_steering_indicator(image, steering_value, radius = 50,text_offset = [10,-5],text_size = 0.8):
    angle_rad = np.interp(steering_value, [-1, 0, 1], [np.pi, np.pi/2, 0])
    center_x, center_y = image.shape[1] // 2, int(image.shape[0] * 0.9)
    end_x = int(center_x + radius * np.cos(angle_rad))
    end_y = int(center_y - radius * np.sin(angle_rad))
    cv2.ellipse(image, (center_x, center_y), (radius, radius), 180, 180, 0, 255, 3)
    cv2.line(image, (center_x, center_y), (end_x, end_y), (0, 255, 0), 3)
    text_offset_x, text_offset_y = text_offset[0], text_offset[1]
    text_x = end_x + text_offset_x
    text_y = end_y + text_offset_y
    text = f'Steering: {steering_value:.2f}'
    cv2.putText(image, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, text_size, (0, 0, 255), 2)
    cv2.circle(image, (end_x, end_y), radius//10, (60, 200, 0), -1)
    cv2.circle(image, (center_x, center_y), radius//15, (255, 255, 255), -1)
    return image

In [24]:
  
def is_port_in_use(port):
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

exe_path = "C:/Users/Acer/Desktop/Research/DonkeySimWin1/donkey_sim.exe"
port = 9091 

cam = (512, 512, 3) 
conf = {"exe_path": exe_path, "port": port, "cam_resolution" :cam,'cam_config':{'img_w': cam[0], 'img_h': cam[1], 'img_d': cam[2]} }
 
if not is_port_in_use(port):
    env = gym.make("donkey-generated-roads-v0", conf=conf)
    obs = env.reset() 
else:
    print(f"Simulator is already running on port {port}. Close the simulator and re run")


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 8.0
Setting default: frame_skip 1
Setting default: log_level 20
Setting default: host localhost
Setting default: steer_limit 1.0
Setting default: throttle_min 0.0
Setting default: throttle_max 1.0
donkey subprocess started


INFO:gym_donkeycar.core.client:connecting to localhost:9091 


loading scene generated_road


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:done sending cam config. {'img_w': 512, 'img_h': 512, 'img_d': 3}
INFO:gym_donkeycar.envs.donkey_sim:sim started!


**CNN**

In [39]:
model_CNN = load_model('CNN_model.h5', compile=False)

In [40]:
throttle_speed = 0.1
steering_temp = 0
steering_amplifier = 5
try:
    while True:
        action = np.array([steering_temp, throttle_speed]) 
        obs, reward, done, info = env.step(action)
        img = cv2.cvtColor(obs,cv2.COLOR_RGB2BGR)
        img = cv2.GaussianBlur(img,  (3, 3), 0)
        img = cv2.resize(img, (512, 512))
        img = cv2.putText(img, str(steering_temp), (50, 50), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 0, 0), 1, cv2.LINE_AA)
        img = draw_steering_indicator(img, steering_temp, radius = 100)
        temp_img = preprocessing(obs) * steering_amplifier
        temp_img = np.expand_dims(temp_img,axis = 0)
        temp = float(model_CNN.predict(temp_img))
        steering_temp = temp * 1.4
        cv2.imshow("window" ,img)
        k = cv2.waitKey(10)
        if k == ord('e'):
            steering_temp = 0
            action = np.array([0, 0])
            break
except KeyboardInterrupt:
    pass

cv2.destroyAllWindows()
obs = env.reset()
 

In [19]:
env.close()

closing donkey sim subprocess
